# Projet IML 
Membres : Paul Galand, Alexandre James, Temano Frogier, Antoine Aubin

## Objectif
Notre objectif est de créer une IA capable de reconnaitre les symbôles du jeu Dobble. 

## Classificateur d'Images
### Entraîner sur des images brutes
Avant de nous baser sur des caractéristiques extraites des images, nous voulons tester la performance d'un classificateur entrainé sur des images brutes.
- Les images sont au format RGB
- Il s'agit de n'importe quel symbole du jeu Dobble, dans n'importe quelle rotation
- Nous nous sommes permis de redimensionner les images pour faciliter l'apprentissage

### Procédé
#### Augmenter le nombre d'images
Le dataset donné ne contenant pas suffisament d'images (5 de chaque symboles, dont 2 que nous réservons pour la prédiction), nous augmentons le nombre d'images en entraînement de quelques milliers en effectuant une rotation aléatoire d'une image existante.

Comme nous nous soucions de la répartition pour entrainer uniformément les données, cette fonction possède une matrice de visite empêchant un symbole d'etre trop ou trop peu dupliqué. Cette matrice est réinitialisée une fois toutes les images originales rencontrés aléatoirement.

Voici la répartition des images par type de symbole:
![](https://i.imgur.com/RTCikGD.png)

#### Redimensionner les images
Toutes nos images n'ont pas les memes dimensions, ce qui est problématique pour le classificateur qui a un nombre statique de neurones en entrée.

Ainsi, nous avons construit une fonction permettant de redimensionner nos images dans un carré de 60x60 pixels, tout en gardant le meme ratio d'aspect.

### Résultats obtenus
#### Précisions

Classifier | SVC | LinearSVC
:---: | :---: | :---:
Temps d'entraînement (m) | 16.27 | 4.01
Précision de la prédiction | **0.842** | 0.798

Les résultats obtenus sont supérieurs à nos attentes étant donné qu'aucun traitement préalable n'est fait sur les images en entrée.

Cependant la précision reste insuffisante, c'est pourquoi nous allons désormais utiliser des classificateurs se basants sur des caractéristiques extraites depuis les images.

#### Cas d'erreurs
![](https://i.imgur.com/Lk8lO7p.png)
![](https://i.imgur.com/qIMz0b0.png)
![](https://i.imgur.com/oZroN2P.png)
![](https://i.imgur.com/jdjD8qw.png)


## Histogramme couleur
### Diminution du nombre de couleur
Notre objectif est de diminuer le nombre de couleur présent sur nos images pour pouvoir ensuite plus facilement les comparer. 

Pour avoir en une image toutes les couleurs présentes dans le jeu Dobble nous concatenons tous les types de symbole du jeu dans un "poster" à partir duquel nous trouvons les couleurs les plus représentatives grâce à Kmeans. 

Nous entrainons un classifier avec les histogrammes obtenus à partir de nos images d'entrainement et nous le testons avec les histogrammes de nos images de tests. 

Le set de test contient 2 images de chaque classe. 
Le set d'entrainement contient 3 images de chaque classe. 

### Résultats obtenus
Nombre de couleurs | 5 | 10 | 15 
:---: | :---: | :---: | :---: 
Images recolorisés et leurs histogrammes | ![](https://i.imgur.com/cUQlnDG.png) | ![](https://i.imgur.com/8E4PATX.png) | ![](https://i.imgur.com/C8ilav4.png) |

**Précision sans classifier**
Nous prenons une image de chaque symbole et réduisons ses couleurs pour déduire un histogramme des couleurs qui y sont présentes.

Lorsque nous prenons une image inconnu pour trouver son label il nous suffit de diminuer ses couleurs, faire son histogramme et le comparer à ceux que nous avons calculer précédemment pour lesquels nous connaissons le label. Nous déduisons le label de l'image inconnu en prenant celui de l'histogramme dont elle est le plus proche. 

Nombre de couleurs | 5 | 8 | 9 | **10** | 11 | 12 | 15
:---: | :---: | :---: | :---: | :---: | :---: | :---: | :---:
Précision | 0.930 | 0.947 | 0.938 | **0.965** | 0.965 | 0.947 | 0.964

La différence entre les résultats est minime. Nous pouvons voir que le maximum de précision est atteint avec 10 et 11 couleurs et que même en augmentant le nombre de couleur le résultat reste très proche. 

Pour la suite nous utiliserons uniquement des réductions en 10 couleurs. 

**Précision avec les classifiers**

Classifier | SVC | LinearSVC | KN (3 neighbors) | KN (1 neighbor) | RandomForest (100 estimators) | Dummy
:---: | :---: | :---: | :---: | :---: | :---: | :---: 
Précision | 0.947 | 0.518 | 0.904 | **0.974** | 0.974 | 0.02

L'usage de classifier ne nous permet pas d'améliorer de façon significative notre précision. Dans le cas de la random forest nos résultats sont entre 0.92 et 0.974 selon le nombre d'estimateur. Nous pouvons conclure que dans ce cas des histogrammes couleurs utiliser un classifier améliore peu les résultats. 


#### Cas d'erreurs 
 ![](https://i.imgur.com/AgRgcpF.png) | ![](https://i.imgur.com/vLkxEpO.png)
 :---: | :---:

Les erreurs rencontrés sont dû à des couleurs identiques entre les symbôles. En prenant en compte la forme nous pourrions ne plus avoir ces erreurs. 


## Descripteur de forme
Identifier le descripteur de forme le plus efficace / utile dans notre cas



### Moments géometriques
Le set de test contient 1 images de chaque classe. 
Le set d'entrainement contient 4 images de chaque classe.


![](https://i.imgur.com/pMNHBK0.png) | ![](https://i.imgur.com/vveQzsn.png) 
:---: | :---: 

**Précision avec les classifiers**

Classifier |  Dummy | LinearSVC | KNeighbors | SVC | RandomForest |
:---: | :---: | :---: | :---: | :---: | :---: |
Précision | 0.018 | 0.280 | 0.421 | 0.851 | **0.991** |

### Topological and geometrical features
Mettre des métriques des résultats en utilisant uniquement la forme pour trouver l'objet.

L'objectif ici est de pouvoir faire de la reconnaissance de forme en s'intéressant aux features topologiques et géométriques.

Nous avons pu extraire les features suivantes:

- Nombre de trous
- Nombre de composants
- Nombre d’Euler
- Superficie
- Nombre de face du contour convexe
- Skeleton

Quelques exemples de skeletons:
![](https://i.imgur.com/sOHxVKw.png)


Puis, nous avons ramené les toutes les features à un float entre 0 et 1 (-1 et 1 pour le nombre d'euler) en les divisant par la valeur max de leur feature respective. Cela afin d'éviter d'avoir des écarts trop grands pour deux mêmes images.

De plus, nous avons effectué un gaussian blur sur les skeletons afin d'élargir les lignes du squelette et ainsi éviter les décalages de pixels.

Enfin nous avons combiné toutes ces features en early fusion et testé avec différents classifieurs.

**Précision avec les classifiers**

Classifier | SVC | LinearSVC | KN (3 neighbors) | KN (1 neighbor) | RandomForest (100 estimators) | Dummy
:---: | :---: | :---: | :---: | :---: | :---: | :---: 
Précision | 0.789 | 0.280 | 0.649 | 0.807 | **0.912** | 0.02

Nous avons également testé le classifieur RandomForest avec toutes les features sauf le skeleton cette fois-ci et nous avions un score de 0.5. On peut donc noter que l'ajout seul du skeleton est un point fort dans la reconnaissance de forme.



## Fusion de charactéristiques
### Early fusion
Pour la early fusion nous avons fais le choix d'unir nos charactéristiques de couleurs avec nos moments géométriques (Hu) grace a une concatenation des caracteristiques. Puisque nous avons 10 informations dans nos histogrammes de couleur, et 7 moments geometriques, nous avons juger que l'equilibre entre les deux tailles etait suffisant pour ne pas avoir besoin de complexifier la concatenation. Voici les résultats que nous avons obtenus :

 Classifier | Dummy | KNeighbors | SVC | Linear SVC | RandomForest |
 :---: | :---: | :---: | :---: | :---: | :---: |
 Précision | 0.018 | 0.5 | 0.965 | 0.974 | 1 |


### Late fusion
Pour la fusion tardive des charactèristiques nous avons choisi de combiner notre descripteur de forme avec Hu Moments et nos histogrammes de couleurs. 

Nous avons choisit un KNeighbors classifier avec 5 voisins pour l'histogramme couleur car ce classifier permet d'avoir un score de detection répartit entre 3 et 5 choix en ayant pendant nos tests toujours la bonne réponse parmis ces choix. L'utilisation d'un KNeighbors avec 1 seul voisin peut fausser nos résultat lors de la fusion car le choix même lorqu'il est faux a un score de 1 lui faisant gagner trop de force par rapport aux scores de notre classifier avec descripteur de forme. 

Pour le classifier du descripteur de forme nous restons sur le RandomForest avec 4 estimateurs qui nous donne des résultats juste dans 100% des cas. 

**Bilan** : Nous obtenons un score de **100% de précision** avec nos tests. 

## Travail de chacun 
Alexandre | Antoine | Paul | Temano
 :---: | :---: | :---: | :---:
Classifier d'images | Histogrammes couleurs | Getting datasets | Descripteur de forme - Extraction des features Topologiques et géométriques
Augmentation du dataset | Late fusion | Hu moments | Early fusion des features Topologiques et géométriques
Vidéo | / | Early and late fusion | /